In [ ]:
import os, csv
import rpy2.robjects as robjects
import plotly.graph_objects as go
import plotly.io

plotly.io.orca.config.executable='/usr/local/bin/orca'

%load_ext rpy2.ipython

In [ ]:
%%R
library(gplots);library(readr);library(ggplot2); library(DESeq2); library(dplyr); library(ggrepel)

In [ ]:
%%R
counts <- read.csv('./merged_gene_counts.txt',
                           sep              = ",",
                           header           = TRUE, 
                           stringsAsFactors = FALSE, 
                           check.names      = FALSE);
rownames(counts) <- counts[,1]
counts <- counts[, -c(1:2)]
counts <- counts[ , order(names(counts))]
sampleTable <- read.csv('./metadata.csv', row.names = 1)
sampleTable <- sampleTable[ order(row.names(sampleTable)), ]
condition <- unlist(sampleTable['Type'])

countdata <- as.matrix(counts)[, colnames(counts) %in% rownames(sampleTable)]

dds <- DESeqDataSetFromMatrix(countData=countdata, colData=sampleTable, design= as.formula(paste("~", 'Type')))

dds <- DESeq(dds)

rld <- rlogTransformation(dds)

suppressMessages(library(RColorBrewer))
(mycols <- brewer.pal(8, "Dark2")[1:length(unique(condition))])
sampleDists <- as.matrix(dist(t(assay(rld))))

jpeg('./output.jpg')
heatmap.2(as.matrix(sampleDists), key=F, trace="none",
          col=colorpanel(100, "black", "white"),
          ColSideColors=mycols[condition], RowSideColors=mycols[condition],
          main="Title",
         margins=c(12,12),
         srtCol=45,
         srtRow=45)
dev.off()